# Packing problem with palletizing

In [11]:
from pandas import DataFrame, read_csv
import os
from os import listdir
from os.path import isfile, join
import vtk #graphics
import math
import numpy as np
import random
import copy

In [12]:
#?minimize

In [13]:
file_list = [f for f in listdir(os.getcwd()) if isfile(f) and ('csv' in f)]
file_list

['1.csv',
 '10.csv',
 '100.csv',
 '101.csv',
 '102.csv',
 '103.csv',
 '104.csv',
 '105.csv',
 '106.csv',
 '107.csv',
 '108.csv',
 '109.csv',
 '11.csv',
 '110.csv',
 '111.csv',
 '112.csv',
 '113.csv',
 '114.csv',
 '115.csv',
 '116.csv',
 '117.csv',
 '118.csv',
 '119.csv',
 '12.csv',
 '120.csv',
 '121.csv',
 '122.csv',
 '123.csv',
 '124.csv',
 '125.csv',
 '126.csv',
 '127.csv',
 '128.csv',
 '129.csv',
 '13.csv',
 '130.csv',
 '131.csv',
 '132.csv',
 '133.csv',
 '134.csv',
 '135.csv',
 '136.csv',
 '137.csv',
 '138.csv',
 '139.csv',
 '14.csv',
 '140.csv',
 '141.csv',
 '142.csv',
 '143.csv',
 '144.csv',
 '145.csv',
 '146.csv',
 '147.csv',
 '148.csv',
 '149.csv',
 '15.csv',
 '150.csv',
 '151.csv',
 '152.csv',
 '153.csv',
 '154.csv',
 '155.csv',
 '156.csv',
 '157.csv',
 '158.csv',
 '159.csv',
 '16.csv',
 '160.csv',
 '161.csv',
 '162.csv',
 '163.csv',
 '164.csv',
 '165.csv',
 '166.csv',
 '167.csv',
 '168.csv',
 '169.csv',
 '17.csv',
 '170.csv',
 '171.csv',
 '172.csv',
 '173.csv',
 '174.csv',
 '

In [14]:
def loadData(file_name):
    data = read_csv(os.getcwd()+'/'+file_name, sep=',',header=1,index_col=False)
    return data

In [15]:
file_name = 'COS-TEST-1.csv'
data = loadData(file_name)
data.head(5)

,SKU,Quantity,Length,Width,Height,Weight,Strength,Aisle,Caustic
0,900001,6,600,400,200,12000,4,1,0
1,900002,5,200,400,200,5000,4,2,0


In [16]:
item = data.iloc[0]
item

SKU         900001
Quantity         6
Length         600
Width          400
Height         200
Weight       12000
Strength         4
Aisle            1
Caustic          0
Name: 0, dtype: int64

In [17]:
class Box():
    #конструктор создает коробку с SKU и начальными параметрами. Изначальная ротация = 3
    def __init__(self, SKU, length, width, height, weight, strength, aisle, caustic):
        self.SKU = SKU
        self.size = np.array([length,width,height])
        self.coords = self.size
        self.rotation = 3
        self.weight = weight
        self.strength = strength
        self.aisle = aisle
        self.caustic = caustic
        self.attPoint = np.array([-1,-1,-1])
        self.farCorner = np.array([-1,-1,-1])
        
    #private функция возвращает матрицу поворота 1-6
    def __getRotationMatrix(self, rotation):
        if rotation == 1:
            return np.array([[1,0,0],[0,0,1],[0,1,0]])
        elif rotation == 2:
            return np.array([[0,0,1],[1,0,0],[0,1,0]])
        elif rotation == 3:
            return np.array([[1,0,0],[0,1,0],[0,0,1]])
        elif rotation == 4:
            return np.array([[0,0,1],[0,1,0],[1,0,0]])
        elif rotation == 5:
            return np.array([[0,1,0],[1,0,0],[0,0,1]])
        elif rotation == 6:
            return np.array([[0,1,0],[0,0,1],[1,0,0]])
    
    #метод позволяющий повернуть коробку в положение 1-6 (3 положение по умолчанию)
    def setRotation(self, rotation):
        self.rotation = rotation
        self.coords = np.dot(self.size, self.__getRotationMatrix(rotation))
        self.farCorner = self.attPoint+self.coords
    
    #вернуть реальные координаты в свой СО
    def getCoords(self):
        return self.coords
    
    #вернуть SKU
    def getSKU(self):
        return self.SKU
    
    #вернуть длину (x по умолчанию)
    def getLength(self):
        return self.size[0]
    
    #вернуть ширину (y по умолчанию)
    def getWidth(self):
        return self.size[1]
    
    #вернуть высоту (z по умолчанию)
    def getHeight(self):
        return self.size[2]
    
    #вернуть свои исходные размеры
    def getSize(self):
        return list(self.size)
    
    #вернуть свое текущее положение
    def getRotation(self):
        return self.rotation
    
    def getWeigth(self):
        return self.weight
    
    def getStrength(self):
        return self.strength
    
    def getAisle(self):
        return self.aisle
    
    def getCaustic(self):
        return self.caustic
    
    def __repr__(self):
        return 'Box(SKU=%s, rot=%s, coords=%s, attPoint=%s, farCorner=%s)' % (self.SKU, self.rotation, self.coords,self.attPoint, self.farCorner)
    
    def displace(self, dcoords):
        self.attPoint = np.array(dcoords)
        self.farCorner = self.attPoint+self.coords
        
    def getAttPoints(self):
        p1 = self.attPoint+self.coords*[1,0,0]
        p2 = self.attPoint+self.coords*[0,1,0]
        p3 = self.attPoint+self.coords*[1,1,0]
        p4 = self.attPoint+self.coords*[0,0,1]
        x = list([list(p1),list(p2),list(p3),list(p4)])
        return x
    
    def getAttPoint(self):
        return list(self.attPoint)
    
    def getFarCorner(self):
        return list(self.farCorner)
    
    def getBottomCenterMass(self):
        return ((self.farCorner[0]+self.attPoint[0]) / 2.0 , (self.farCorner[1]+self.attPoint[1]) / 2.0, self.attPoint[2])
    
    def getMaxZ(self):
        return self.farCorner[2]
    
    def getVolume(self):
        return self.size[0]*self.size[1]*self.size[2]
    
    def getUpperPlane(self):
        return ((self.attPoint[0],self.attPoint[1], self.farCorner[2]), (self.farCorner[0],self.farCorner[1], self.farCorner[2]))
    
    def getPointsToCheck(self):
        #не использовать, вместо него проверка центра масс
        return ((self.attPoint[0], self.attPoint[1], self.attPoint[2]),
                (self.attPoint[0], self.farCorner[1], self.attPoint[2]),
                (self.farCorner[0], self.farCorner[1], self.attPoint[2]),
                (self.farCorner[0], self.attPoint[1], self.attPoint[2]))
    
    def getAnswer(self):
        return [self.SKU, 
                self.attPoint[0], self.attPoint[1], self.attPoint[2], 
                self.farCorner[0], self.farCorner[1], self.farCorner[2],
                self.aisle,
                self.weight
               ]
    
    def __eq__(self, other):
        """Override the default Equals behavior"""
        if (self.getSKU() == other.getSKU() and
           self.getRotation() == other.getRotation() and
           self.getAttPoint() == other.getAttPoint() and
           self.getSize() == other.getSize()):
            return True
        else:
            return False
    
    def __ne__(self, other):
        if  not (self.getSKU() == other.getSKU() and
           self.getRotation() == other.getRotation() and
           self.getAttPoint() == other.getAttPoint() and
           self.getSize() == other.getSize()):
            return True
        else:
            return False

In [18]:
def getSortedAttPoints(attpoints):
    x = attpoints[:]
    def point_is_better(p1, p2):
        def getDistance(point):
            return math.sqrt(point[0] ** 2 + point[1] ** 2 + point[2] ** 2)

        def evristics(p1,p2):
            if p1[0]<p2[0]:
                return 1
            else:
                return -1

        if p1[2] < p2[2]:
            return 1
        elif p1[2] > p2[2]:
            return -1
        elif getDistance(p2)>getDistance(p1):
            return 1
        elif getDistance(p1)>getDistance(p2):
            return -1
        else:
            return evristics(p1,p2)
    x.sort(cmp=point_is_better, reverse=True)
    return x
    

In [19]:
#Проверка на пересечение объемов
def theyMerge(b1,b2):
    def getOverlap(a, b):
        return max(0, min(a[1], b[1]) - max(a[0], b[0]))

    def plane1(b1,b2):
        l_x_1 = (b1.attPoint[0], b1.farCorner[0])
        l_x_2 = (b2.attPoint[0], b2.farCorner[0])
        l_z_1 = (b1.attPoint[2], b1.farCorner[2])
        l_z_2 = (b2.attPoint[2], b2.farCorner[2])

        return getOverlap(l_x_1, l_x_2)*getOverlap(l_z_1, l_z_2)

    def plane2(b1,b2):
        l_x_1 = (b1.attPoint[0], b1.farCorner[0])
        l_x_2 = (b2.attPoint[0], b2.farCorner[0])
        l_y_1 = (b1.attPoint[1], b1.farCorner[1])
        l_y_2 = (b2.attPoint[1], b2.farCorner[1])
        #print l_x_1, l_x_2
        #print l_y_1, l_y_2
        return getOverlap(l_x_1, l_x_2)*getOverlap(l_y_1, l_y_2)

    def plane3(b1,b2):
        l_z_1 = (b1.attPoint[2], b1.farCorner[2])
        l_z_2 = (b2.attPoint[2], b2.farCorner[2])
        l_y_1 = (b1.attPoint[1], b1.farCorner[1])
        l_y_2 = (b2.attPoint[1], b2.farCorner[1])
        #print l_z_1, l_z_2
        #print l_y_1, l_y_2
        #tuple sintead of list, mb faster
        return getOverlap(l_z_1, l_z_2)*getOverlap(l_y_1, l_y_2)
    
    if plane1(b1,b2)*plane2(b1,b2) > 0 or plane2(b1,b2)*plane3(b1,b2) > 0 or plane1(b1,b2)*plane3(b1,b2)>0:
        return True
    else:
        return False

In [20]:
data

,SKU,Quantity,Length,Width,Height,Weight,Strength,Aisle,Caustic
0,900001,6,600,400,200,12000,4,1,0
1,900002,5,200,400,200,5000,4,2,0


In [21]:
def getInitialList(data):
    ls = []
    for each in range(len(data)):
        count = data["Quantity"][each]
        while count > 0:
            ls.append(Box(data['SKU'][each], data['Length'][each], data['Width'][each], data['Height'][each], data['Weight'][each],data['Strength'][each],data['Aisle'][each],data['Caustic'][each]))
            count-=1
    return ls

In [22]:
def pointBelongs(point, plane):
    if (point[2] == plane[0][2] and
        point[0] <= plane[1][0] and
        point[0] >= plane[0][0] and
        point[1] <= plane[1][1] and
        point[1] >= plane[0][1]):
        return True
    else:
        return False  

In [23]:
def decode(chromosome):
    #изначально только 0 0 0 точка
    attPoints = [[0,0,0]]
    chromosome[0].displace(attPoints.pop(0)) #ставим первый элемент в угол, удаляем точку
    attPoints += chromosome[0].getAttPoints() #добавляем точки присоединения
    attPoints = getSortedAttPoints(attPoints) #сортируем согласно правилам точки
    status = [0, 0, 0]
    
    for i in range(1,len(chromosome)):
        displaced = False
        for p_num, point in enumerate(attPoints):
        #берем следующую точку и блок, ставим его и проверяем что:
            chromosome[i].displace(point)

            #центр масс не свисает
            
            if chromosome[i].getAttPoint()[2] == 0: #ака лежит на полу то все ок:
                itDoesntHang = True
            else:
                for j in range(i):
                    itDoesntHang = False
                    if pointBelongs(chromosome[i].getBottomCenterMass(), chromosome[j].getUpperPlane()):
                        itDoesntHang = True
                        break;

            #не выходит за границы данной плоскости палеты [0] (x) < pallet_x [3] && (z) < pallet_z
            if chromosome[i].farCorner[0] <= pallet_size[0] and chromosome[i].farCorner[1] <= pallet_size[1]:
                itFits = True
            else:
                itFits = False

            #блоки не налагаются друг на друга в объеме
            theyDontMerge = True
            for j in range(i):
                if theyMerge(chromosome[i], chromosome[j]):
                    theyDontMerge = False
                    break

            if theyDontMerge and itFits and itDoesntHang:
                #print 'YAY'
                attPoints.pop(p_num)
                displaced = True
                attPoints += [x for x in chromosome[i].getAttPoints() if x not in attPoints] #добавляем точки присоединения
                attPoints = getSortedAttPoints(attPoints) #сортируем согласно правилам точки
                break
        if not displaced:
            return 0 #Houston we have problems here
    return 1 #success
            #print(attPoints)

      itDoesntHang = True
            #блок лежит на дне или других блоках и не свисает
            if chromosome[i].attPoint[2] == 0:
                itDoesntHang = True
            else:
                points_to_check = chromosome[i].getPointsToCheck()
                #print points_to_check
                points_status = [False, False, False, False]
                for j in range(i):
                    for point_num, each in enumerate(points_to_check):
                        if pointBelongs(points_to_check[point_num], chromosome[j].getUpperPlane()):
                            points_status[point_num] = True
                if False in points_status:
                    itDoesntHang = False

In [24]:
def getMaxZ(chromosome):
        max_z = chromosome[0].farCorner[1] #максимальная высота первого блока дальней точки
        for each in chromosome[1:]:
            if each.getMaxZ()>max_z:
                max_z = each.getMaxZ()
        return max_z

In [25]:
def getPercolation(chromosome):
        sum_volume = 0.
        for each in chromosome:
            sum_volume += each.getVolume()
            
        max_volume = getMaxZ(chromosome) * pallet_size[0] * pallet_size[1]
        return sum_volume / max_volume

In [26]:
def fitness_function(chromosome):
    return getPercolation(chromosome)  

# Генетический алгоритм

In [27]:
##### настройки
pallet_size = (1200,800) #z,x: y - верх.
population_size = 10
initial_mutation = 0.3
max_deep = 20

In [28]:
def shuffle(chromosome, beta):
    x = copy.deepcopy(chromosome)
    for i in range(len(chromosome)):
        x.append(x.pop(random.randint(0, len(chromosome)-1)))
        if random.random() < beta:
            x[random.randint(0, len(chromosome)-1)].setRotation(random.randint(1,6))
    return x

In [29]:
def improved(chromosome):
    for i in range(3):
        j = random.randint(0, len(chromosome)-1)
        rot = random.randint(1,6)
        while rot == chromosome[j].getRotation():
            rot = random.randint(1,6)
        chromosome[j].setRotation(rot)
    
    return decode(chromosome)

In [30]:
#функция для сортировки популяции
def compare(chr1, chr2):
    return int((fitness_function(chr2)-fitness_function(chr1))*100)


In [31]:
def crossingover(chromosome1, chromosome2):
    p = random.randint(0, len(chromosome1)-2)
    q = random.randint(1, len(chromosome1)-1-p)
    prohibited = [[0,len(chromosome1)-1]]
    need_to_do = True
    while need_to_do:
        while [p,q] in prohibited:
            p = random.randint(0, len(chromosome1)-2)
            q = random.randint(1, len(chromosome1)-1-p)

        #print p, p+q

        ans = copy.deepcopy(chromosome1[p:p+q+1])
        the_rest = copy.deepcopy(chromosome2[:])
        for each in ans:
            need_to_find = True
            num = 0
            while need_to_find:
                if each.getSKU() == the_rest[num].getSKU():
                    the_rest.pop(num)
                    need_to_find = False
                else:
                    num += 1
        ans += the_rest
        if decode(ans) and ans not in population:
            need_to_do = False
        else:
            prohibited.append([p,q])
    return ans

In [32]:
def mutation(chromosome, alpha):
    x = copy.deepcopy(chromosome[:])
    prob = random.random()
    if prob < alpha:
        x[random.randint(0, len(chromosome)-1)].setRotation(random.randint(1,6))
        need_to_mutate = True
        while need_to_mutate:
            if decode(x) and x not in population:
                need_to_mutate = False
            else:
                x[random.randint(0, len(chromosome)-1)].setRotation(random.randint(1,6))
    return x

In [33]:
def random_shuffle(chromosome):

    need_to_shuffle = True
    while need_to_shuffle:
        temp = shuffle(chromosome, initial_mutation )
        if decode(temp):
            need_to_shuffle = False
    return temp
            
            

In [34]:
%time
#делаем начальную популяцию без клонов
population = []
initialList = getInitialList(data)
while len(population) < population_size:
    temp = shuffle(initialList, initial_mutation ) #убираем клонов
    while temp in population:
        temp = shuffle(initialList, initial_mutation )
    if decode(temp):
        population.append(temp)
    elif improved(temp):
        population.append(temp)

population.sort(cmp = compare)
for each in population:
    print decode(each), fitness_function(each)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 5.96 µs
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.479166666667
1 0.479166666667


In [35]:
population[0]

[Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[0 0 0], farCorner=[600 400 200]),
 Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[  0 400   0], farCorner=[600 800 200]),
 Box(SKU=900002, rot=6, coords=[200 200 400], attPoint=[600   0   0], farCorner=[800 200 400]),
 Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[600 200   0], farCorner=[1200  600  200]),
 Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[  0   0 200], farCorner=[200 400 400]),
 Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[200   0 200], farCorner=[400 400 400]),
 Box(SKU=900001, rot=5, coords=[400 600 200], attPoint=[600 200 200], farCorner=[1000  800  400]),
 Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[  0 400 200], farCorner=[200 800 400]),
 Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[  0   0 400], farCorner=[600 400 600]),
 Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[400   0 200], farCorner=[600 400 400]),
 Box(SKU=900001, rot=5, coords=[400 600 

In [36]:
fitness_function(population[0])

0.6388888888888888

In [37]:
for each in population:
    print decode(each), fitness_function(each)
population

1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.479166666667
1 0.479166666667


[[Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[0 0 0], farCorner=[600 400 200]),
  Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[  0 400   0], farCorner=[600 800 200]),
  Box(SKU=900002, rot=6, coords=[200 200 400], attPoint=[600   0   0], farCorner=[800 200 400]),
  Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[600 200   0], farCorner=[1200  600  200]),
  Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[  0   0 200], farCorner=[200 400 400]),
  Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[200   0 200], farCorner=[400 400 400]),
  Box(SKU=900001, rot=5, coords=[400 600 200], attPoint=[600 200 200], farCorner=[1000  800  400]),
  Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[  0 400 200], farCorner=[200 800 400]),
  Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[  0   0 400], farCorner=[600 400 600]),
  Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[400   0 200], farCorner=[600 400 400]),
  Box(SKU=900001, rot=5, coord

In [38]:
count = 0
while count <= max_deep and fitness_function(population[0]) != 1:
    
    count +=1
    for i in range(len(population)-1, 0, -1):
        #update - пробую скрещивать случайно, вместо сильного со всеми
        rand_int_1 = random.randint(0,len(population)-1)
        rand_int_2 = random.randint(0,len(population)-1)
        cross_product = crossingover(population[rand_int_1],population[rand_int_2])
        if fitness_function(cross_product) > fitness_function(population[-1]):
            population.pop(-1)
            population.append(cross_product)
        
        rand_int = random.randint(0,len(population)-1)
        population[rand_int] = mutation(population[rand_int], initial_mutation)
        
        rand_value = random.random()
        if rand_value < 0.2:
            population[rand_int] = random_shuffle(population[rand_int])
    print count  
    population.sort(cmp=compare)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [39]:
for each in population:
    print decode(each), fitness_function(each)
population

1 0.958333333333
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889
1 0.638888888889


[[Box(SKU=900001, rot=2, coords=[400 200 600], attPoint=[0 0 0], farCorner=[400 200 600]),
  Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[  0 200   0], farCorner=[600 600 200]),
  Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[600 200   0], farCorner=[800 600 200]),
  Box(SKU=900002, rot=4, coords=[200 400 200], attPoint=[800 200   0], farCorner=[1000  600  200]),
  Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[  0 200 200], farCorner=[600 600 400]),
  Box(SKU=900001, rot=1, coords=[600 200 400], attPoint=[400   0   0], farCorner=[1000  200  400]),
  Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[600 200 200], farCorner=[1200  600  400]),
  Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[1000    0    0], farCorner=[1200  400  200]),
  Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[1000  400    0], farCorner=[1200  800  200]),
  Box(SKU=900002, rot=2, coords=[400 200 200], attPoint=[  0 600   0], farCorner=[400 800 200]),
  Box(SKU=90000

# Вывод хромосомы с решением

In [40]:
chromosome = population[0]

In [41]:
packed_items = []
total_weight = 0
for each in chromosome:
    packed_items.append(each.getAnswer())
    total_weight += each.getWeigth()
answer = DataFrame(packed_items)
answer.columns = ['SKU','x_1^i','y_1^i','z_1^i','x_2^i','y_2^i','z_2^i','Aisle','Weight']
print getMaxZ(chromosome), getPercolation(chromosome), total_weight
answer

400 0.958333333333 97000


,SKU,x_1^i,y_1^i,z_1^i,x_2^i,y_2^i,z_2^i,Aisle,Weight
0,900001,0,0,0,400,200,600,1,12000
1,900001,0,200,0,600,600,200,1,12000
2,900002,600,200,0,800,600,200,2,5000
3,900002,800,200,0,1000,600,200,2,5000
4,900001,0,200,200,600,600,400,1,12000
5,900001,400,0,0,1000,200,400,1,12000
6,900001,600,200,200,1200,600,400,1,12000
7,900002,1000,0,0,1200,400,200,2,5000
8,900002,1000,400,0,1200,800,200,2,5000
9,900002,0,600,0,400,800,200,2,5000


In [42]:
f = open(os.getcwd()+'/'+'answer'+file_name, 'wb')
f.write(str(getMaxZ(chromosome)) + ' ' + str(getPercolation(chromosome)) + ' ' + str(total_weight) + '\n')
answer.to_csv(f,index = False)
f.close()

In [43]:
x = 600
y = 400
z = 200

origin_x = 0
origin_y = 0
origin_z = 0

P0 = [origin_x, origin_y, origin_z] 
P1 = [origin_x + x , origin_y, origin_z]
P2 = [origin_x + x, origin_y + y, origin_z]
P3 = [origin_x, origin_y + y, origin_z]
P4 = [origin_x,  origin_y, origin_z + z]
P5 = [origin_x + x,  origin_y, origin_z + z]
P6 = [origin_x + x, origin_y + y, origin_z + z]
P7 = [origin_x,  origin_y + y, origin_z + z]

# Create the points
points = vtk.vtkPoints()
points.InsertNextPoint(P0)
points.InsertNextPoint(P1)
points.InsertNextPoint(P2)
points.InsertNextPoint(P3)
points.InsertNextPoint(P4)
points.InsertNextPoint(P5)
points.InsertNextPoint(P6)
points.InsertNextPoint(P7)
 
# Create a hexahedron from the points
hex = vtk.vtkHexahedron() 
hex.GetPointIds().SetId(0,0)
hex.GetPointIds().SetId(1,1)
hex.GetPointIds().SetId(2,2)
hex.GetPointIds().SetId(3,3)
hex.GetPointIds().SetId(4,4)
hex.GetPointIds().SetId(5,5)
hex.GetPointIds().SetId(6,6)
hex.GetPointIds().SetId(7,7)
 
# Add the hexahedron to a cell array
hexs = vtk.vtkCellArray()
hexs.InsertNextCell(hex)
 
# Add the points and hexahedron to an unstructured grid
uGrid = vtk.vtkUnstructuredGrid()
uGrid.SetPoints(points)
uGrid.InsertNextCell(hex.GetCellType(), hex.GetPointIds())
 
# Visualize
mapper = vtk.vtkDataSetMapper()
mapper.SetInputData(uGrid)
 
actor = vtk.vtkActor()
actor.SetMapper(mapper)

axes = vtk.vtkAxesActor()
axes.SetTotalLength([x+100,y+100,z+100])
axes.SetXAxisLabelText('X - ' + str(x))
axes.SetYAxisLabelText('Y - ' + str(y))
axes.SetZAxisLabelText('Z - ' + str(z))

renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
 
renderer.AddActor(actor)
renderer.AddActor(axes)
renderer.SetBackground(.2, .3, .4)
 
renderWindow.Render()
renderWindowInteractor.Start()

In [44]:
chromosome = [Box(900001, 600, 400, 200, 1200, 100, 100, 100),
              Box(900001, 600, 400, 200, 1200, 100, 100, 100),
              Box(900002, 200, 400, 200, 1200, 100, 100, 100),
              Box(900002, 200, 400, 200, 1200, 100, 100, 100),
              Box(900002, 200, 400, 200, 1200, 100, 100, 100),
              Box(900002, 200, 400, 200, 1200, 100, 100, 100),
              Box(900002, 200, 400, 200, 1200, 100, 100, 100),
              Box(900001, 600, 400, 200, 1200, 100, 100, 100),
              Box(900001, 600, 400, 200, 1200, 100, 100, 100),
              Box(900001, 600, 400, 200, 1200, 100, 100, 100),
              Box(900001, 600, 400, 200, 1200, 100, 100, 100)              
]
rot = [3,#
       5,#
       3,#
       3,#
       4,#
       3,#
       2,#
       3,#
       3,#
       3,#
       3]
pos = [[0,0,0],
       [800,0,0],
       [600,0,0],
       [0,400,0],
       [200,400,0],
       [400,400,0],
       [800,600,0],
       [0,0,200],
       [600,0,200],
       [0,400,200],
       [600,400,200]]
for i, each in enumerate(chromosome):
    chromosome[i].displace(pos[i])
    chromosome[i].setRotation(rot[i])
chromosome

[Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[0 0 0], farCorner=[600 400 200]),
 Box(SKU=900001, rot=5, coords=[400 600 200], attPoint=[800   0   0], farCorner=[1200  600  200]),
 Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[600   0   0], farCorner=[800 400 200]),
 Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[  0 400   0], farCorner=[200 800 200]),
 Box(SKU=900002, rot=4, coords=[200 400 200], attPoint=[200 400   0], farCorner=[400 800 200]),
 Box(SKU=900002, rot=3, coords=[200 400 200], attPoint=[400 400   0], farCorner=[600 800 200]),
 Box(SKU=900002, rot=2, coords=[400 200 200], attPoint=[800 600   0], farCorner=[1200  800  200]),
 Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[  0   0 200], farCorner=[600 400 400]),
 Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[600   0 200], farCorner=[1200  400  400]),
 Box(SKU=900001, rot=3, coords=[600 400 200], attPoint=[  0 400 200], farCorner=[600 800 400]),
 Box(SKU=900001, rot=3, coords=[600 4